# Submission of Calculation with Subsequent Results Query

In this demo, we will submit a calculation, check the status of the job and query the results after it is finished.

## Initialization of Workspace 

Specify the Azure quantum workspace and the 'connection string' which allows us to connect to the workspace

In [ ]:
from azure.quantum import Workspace
from azure.quantum.job import JobFailedWithResultsError

# insert connection string from Azure Portal Workspace Access Keys
connection_string = "" 
workspace = Workspace.from_connection_string(connection_string)

In [ ]:
# To submit Accelerated DFT jobs, we will be using the microsoft.dft target in the workspace.
print("Verifying access to Accelerated DFT target.")
target = workspace.get_targets("microsoft.dft")
print("Verification complete.")

## Define Input and Submit Accelerated DFT job

In [ ]:
# First, let's define the molecular structure, loaded from an xyz file.
from pathlib import Path
GeomFile = "molecules/C2H4.xyz"

In [ ]:
# Secondly, let's give a name for the job.
job_name = 'C2H4_fh'

Now we submit the calculations to MADFT service.

In [ ]:
# Next, we create a dictionary variable to specify the parameters for the DFT calculation. 
dft_input_params = {
  "tasks": [
    {
      "taskType": "fh", 
      "basisSet": { "name": 'def2-svpd'},
      "xcFunctional": { "name": "b3lyp", "gridLevel": 4 },
      "molecule": { "charge": 0, "multiplicity": 1 },
      "scf": { "method": "rks", "maxSteps": 100, "convergeThreshold": 1e-8, "requireWaveFunction": True }
      # example with PCM solvent and D3 dispersion correction
      #"scf":{"method":"rks","dispersion":"d3zero","convergeThreshold":1e-8,"pcm":{"solverType":"iefpcm","solvent":"water"}}
    }
  ]
}

# We are now ready to submit the Job using the target.submit call. It takes three parameters-
# 1. The input molecule in xyz format.
# 2. The DFT parameters that we declared above.
# 3. A friendly name to help identify the job in the Azure Portal later.

print("Submitting DFT job.")

job = target.submit(
    input_data=Path(GeomFile).read_text(),
    input_params = dft_input_params,
    name= job_name)
    
print("\nDFT job has been submitted.")
print(f"\nJob name: {job_name}")


Show the status of the job. If the job has finished, read the results of the job

In [ ]:
job.refresh()
print(f'Job: "{job_name}" is {job.details.status}')
if job.details.status == 'Succeeded':
    qcschema = job.get_results()["results"][0]

## Results

The results of the calculation are stored in the QCSchema format dict.

For an FH calculation we can see the hessian by simply looking at the key "return_result".

In [ ]:
#print("Hessian: ",qcschema["return_result"])

Other useful information is stored in the output dict, for example:

In [ ]:
print("Number of Basis Functions: ", qcschema["properties"]["calcinfo_nbasis"])
print("Total Energy (Hartree): ", qcschema["properties"]["return_energy"])
print("Nuclear Repulsion Energy (Hartree): ", qcschema["properties"]["nuclear_repulsion_energy"])
print("Total Calculation Time (s): ", qcschema["provenance"]["total_time_seconds"])

Wavefunction information is also saved in the output if 'requireWavefunction: True' was set.
The "wavefunction" key contains orbitals, orbital energies, orbital occupancies, and Fock matrices.
This will be used in our later examples for property calculations.

## Output to QCSchema json file

Saving to a json-formatted file makes it easy to read/write/visualize the QCSchema key structure.  

In [ ]:
import json
qcschema_json = job_name + "_output.json"
with open(qcschema_json, "w") as fp:
    json.dump(qcschema, fp)

# Property Calculation: Infrared Spectrum

Using the wavefunction information saved in the Accelerated DFT output, we can compute properties.
Here is an example of an IR spectrum.

First we load the required packages and read the Accelerated DFT QCSchema result into the PySCF objects 'ks', 'mol' and 'hessian'

In [ ]:
import pyscf
from tools.libqcschema import *
from pyscf import gto, dft
from pyscf import hessian
from pyscf.hessian.thermo import *
from pyscf.prop import infrared
import json
import numpy as np

# Create DFT object
mol, ks = recreate_scf_obj(qcschema)

# Form Hessian object and Load Hessian from QCSchema dict
hessian = ks.Hessian()
hessian.de = load_qcschema_hessian(qcschema)

We then use the hessian to find the vibrational frequencies and normal modes.

In [ ]:
# Compute Vibrational Frequencies
freq = harmonic_analysis(mol,hessian.de)
dump_normal_mode(mol,freq)

We also (optionally) compute the thermochemistry corrections such as zero point energy. 

In [ ]:
thermochem = thermo(ks,freq['freq_au'], 298.15)
print("")
#print("Thermochem:",thermochem)
for i in thermochem.keys():
    print(i, " ", thermochem[i])

We now have the vibrational frequencies which tells us the positions of the peaks in the IR spectrum. 
We now compute the intensities.

In [ ]:
###############################
##### Compute IR Spectrum #####
###############################

# make IR object and populate with info
ks_ir = prepare_ir(ks,hessian,freq)

# compute IR intensities
infrared.rhf.kernel_dipderiv(ks_ir)
ir_intensity = infrared.rhf.kernel_ir(ks_ir)

# Print summary
ks_ir.summary()

In [ ]:
import matplotlib as plt
fig = ks_ir.plot_ir()[0]
fig.show()
fig.savefig("ir_spectrum_C2H4.png")